In [2]:
import pandas as pd

# Remplacez 'chemin/vers/votre_fichier.tsv' par le chemin de votre fichier
df_recherche_film = pd.read_csv('/Users/julien-thomasfassinot/Documents/WCS/PROJET 2/Data final/Fonctionnel/df_recherche_film.csv')
df_entrainement = pd.read_csv('/Users/julien-thomasfassinot/Documents/WCS/PROJET 2/Data final/Fonctionnel/df_entrainement.csv')
df_entrainement.head(2)


/var/folders/7k/806tk5d91nl8bm1xx68vdntc0000gn/T/ipykernel_10270/666579146.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_recherche_film = pd.read_csv('/Users/julien-thomasfassinot/Documents/WCS/PROJET 2/Data final/Fonctionnel/df_recherche_film.csv')


,tconst,title,production_countries,production_companies_name,acteurs,actrices,realisateurs,producteurs,runtimeMinutes,budget,...,US,FR,IT,AL,ES,Autres,realisateur_hashed,poster_path,overview,overview_fr
0,tt0228556,Mais qu'est-ce que j'ai fait au Bon Dieu pour ...,['FR'],"['Sofracima', 'Les Productions Artistes Associ...","['Robert Castel', 'Michel Boujenah', 'Jacques ...","['Antoinette Moya', 'Lorraine Bracco', 'Claude...",Jan Saint-Hamont,[],90,0,...,0,1,0,0,0,0,333557029846721589575041874025990681763,/kOwKMrReaMm676sZnxV9YtaSmWw.jpg,"Gaby is happy. After eighteen years of work, h...",Gaby est heureux. Après dix-huit ans de travai...
1,tt0066346,"Si je te rencontre, je te tue et pour ça j'ira...",['IT'],"['Minerva Film', 'Rieti Film']","[""Donald O'Brien"", 'Gordon Mitchell', 'Mario B...","['Femi Benussi', 'Pia Giancaro']",Gianni Crea,['Fernando Morbis'],88,0,...,0,0,1,0,0,0,54479597263321372376164868271361071287,/gakeJb5ZldFcT4shRF5Bnre223e.jpg,A farmer returns home from the forest to find ...,Un fermier rentre chez lui après une journée d...


In [3]:
df_recherche_film.head(2)

,tconst,title,startYear,runtimeMinutes,popularity,vote_average,vote_count,actors,actresses,directors,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,tt0011436,L'homme sans nom,1921,\N,1.091,0.0,0,"['Harry Liedtke', 'Paul Otto', 'Jakob Tiedtke'...","['Mady Christians', 'Lori Leux', 'Edith Meller...",['Georg Jacoby'],...,0,0,0,0,0,0,0,0,0,0
1,tt0055747,L'Amour à 20 ans,1962,110,3.770,6.7,41,"['Jean-Pierre Léaud', 'Patrick Auffay', 'Franç...","['Marie-France Pisier', 'Rosy Varte', 'Cristin...","['Shintarô Ishihara', 'Marcel Ophüls', 'Renzo ...",...,0,0,0,0,1,0,0,0,0,0


In [4]:
df_recherche_film = df_recherche_film.rename(columns={"title_FR": "title"})
df_recherche_film.head(2)

,tconst,title,startYear,runtimeMinutes,popularity,vote_average,vote_count,actors,actresses,directors,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,tt0011436,L'homme sans nom,1921,\N,1.091,0.0,0,"['Harry Liedtke', 'Paul Otto', 'Jakob Tiedtke'...","['Mady Christians', 'Lori Leux', 'Edith Meller...",['Georg Jacoby'],...,0,0,0,0,0,0,0,0,0,0
1,tt0055747,L'Amour à 20 ans,1962,110,3.770,6.7,41,"['Jean-Pierre Léaud', 'Patrick Auffay', 'Franç...","['Marie-France Pisier', 'Rosy Varte', 'Cristin...","['Shintarô Ishihara', 'Marcel Ophüls', 'Renzo ...",...,0,0,0,0,1,0,0,0,0,0


In [5]:
df_recherche_film.to_csv("df_recherche_film.csv", index=False)

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.neural_network import MLPRegressor
import numpy as np
import pandas as pd

def find_similar_film_genre(film_name, k=5):
    """
    Trouve les films les plus similaires à un film donné basé sur les genres et la popularité,
    en utilisant un modèle k-NN suivi d'un réseau neuronal.

    Parameters:
    film_name (str): Le nom (ou une partie du nom) du film à rechercher.
    k (int): Le nombre de films similaires à retourner.

    Returns:
    DataFrame: Un DataFrame contenant les films similaires.
    """

    def search_and_select_movie(df, query):
        # Étape 1 : Recherche partielle et insensible à la casse
        query = input("\nEntrez le nom du film : ")
        matches = df_recherche_film[df_recherche_film["title"].str.contains(query, case=False, na=False)]

        # Si aucune correspondance, informer l'utilisateur
        if matches.empty:
            print("Aucun film correspondant trouvé.")
            return None

        # Étape 2 : Afficher les films correspondants
        print("\nFilms correspondants :")
        for idx, row in matches.iterrows():
            print(f"{idx}: {row['title']} ({row['startYear']})")

        # Étape 3 : Laisser l'utilisateur choisir un film
        while True:
            try:
                choice = input("\nEntrez le numéro du film que vous souhaitez sélectionner : ")
                choice = int(choice)
                if choice in matches.index:
                    selected_movie = matches.loc[choice]
                    print(f"\nVous avez sélectionné : {selected_movie['title']} ({selected_movie['startYear']})")
                    return selected_movie
                else:
                    print("Numéro invalide. Essayez encore.")
            except ValueError:
                print("Veuillez entrer un numéro valide.")

    # Recherche du film correspondant
    selected_movie = search_and_select_movie(df_entrainement, film_name)

    # Vérification si un film a été sélectionné
    if selected_movie is None:
        return "Aucun film n'a été sélectionné."

    # Vérification si le film sélectionné est dans le DataFrame
    if selected_movie.empty:
        return f"Le film '{film_name}' n'a pas été trouvé dans le dataset."

    # On utilise uniquement le premier film trouvé (évite les doublons)
    film_recherche = selected_movie

    # On exclut le film recherché de la recherche de voisins
    df_candidates = df_entrainement[df_entrainement['title'] != film_recherche['title']]

    # Caractéristiques pour la similarité
    features = [
        'popularity', 'vote_count',
        'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
        'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
        'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
    ]

    # Standardisation des données
    scaler = StandardScaler()
    X = scaler.fit_transform(df_candidates[features])

    # k-NN : Identification des voisins proches
    knn = NearestNeighbors(n_neighbors=2 * k, metric='euclidean')  # Plus de voisins pour permettre une sélection affinée
    knn.fit(X)

    # Caractéristiques du film recherché
    film_features = scaler.transform(selected_movie[features].values.reshape(1, -1))

    # Trouver les voisins proches
    distances, indices = knn.kneighbors(film_features)
    candidate_indices = indices[0]
    candidates = df_candidates.iloc[candidate_indices]

    # Préparation des données pour le réseau neuronal
    X_candidates = scaler.transform(candidates[features])
    y_similarities = 1 / (1 + distances[0])  # Inverser les distances pour obtenir une similarité

    # Entraînement d'un modèle NN pour ajuster les scores de similarité
    nn = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
    nn.fit(X_candidates, y_similarities)

    # Prédiction des scores pour les voisins
    predicted_scores = nn.predict(X_candidates)

    # Ajouter les scores prédits et trier les résultats
    candidates['similarity_score'] = predicted_scores
    recommended_films = candidates.sort_values(by='similarity_score', ascending=False).head(k)

    return recommended_films[['title','startYear', 'similarity_score','poster_path', 'overview_fr']]

In [9]:
find_similar_film_genre("")


Films correspondants :
20669: Bikini Inception (2015)
30978: Inception (2010)

Vous avez sélectionné : Inception (2010)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/var/folders/7k/806tk5d91nl8bm1xx68vdntc0000gn/T/ipykernel_10270/4166624858.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['similarity_score'] = predicted_scores


,title,startYear,similarity_score,poster_path,overview_fr
20445,Pulp Fiction,1994,0.126265,/d5iIlFn5s0ImszYzBPb8JPIfbXD.jpg,"Un tueur à gages amateur de hamburgers, son pa..."
20442,Fight Club,1999,0.101777,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Un insomniaque à retardement et un vendeur de ...
20424,Forrest Gump,1994,0.096849,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,Un homme au QI faible a accompli de grandes ch...
20439,Matrix,1999,0.081870,/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg,"Se déroulant au 22e siècle, Matrix raconte l'h..."
20478,Titanic,1997,0.080481,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,"Rose DeWitt Bukater, 101 ans, raconte l'histoi..."
